<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [36]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
DEV_DATA_PATH = '/workspace/data/criteo/dac'

DEV_TRAIN_PATH = os.path.join(DEV_DATA_PATH, 'train.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + DEV_TRAIN_PATH)

In [4]:
df = df.sample(False, 0.5)

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

Keeping only first two categorical features for simplicity

In [6]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [7]:
df = df.fillna(0, subset=num_columns)

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler


# Configure an ML pipeline
cat_index_columns = [col_name + "_index" for col_name in cat_columns]
cat_vec_columns = [col_name + "_vec" for col_name in cat_columns]
cat_indexer_0 = StringIndexer(inputCol=cat_columns[0], outputCol=cat_index_columns[0], handleInvalid='keep')
cat_indexer_1 = StringIndexer(inputCol=cat_columns[1], outputCol=cat_index_columns[1], handleInvalid='keep')
ohe_estimator = OneHotEncoderEstimator(
    inputCols=[cat_index_columns[0], cat_index_columns[1]],
    outputCols=[cat_vec_columns[0], cat_vec_columns[1]])

columns_to_assemble = []
columns_to_assemble.extend(num_columns)
columns_to_assemble.extend(cat_vec_columns)

vector_assembler = VectorAssembler(inputCols=columns_to_assemble, outputCol="features")

df_pipeline = Pipeline(stages=[cat_indexer_0, cat_indexer_1, ohe_estimator, vector_assembler])

### Fit and save pipeline

In [9]:
MODEL_PATH = '/workspace/models/criteo/dac'

DF_PIPELINE_PATH = os.path.join(MODEL_PATH, 'df_pipeline')

df_pipeline.fit(df).write().overwrite().save(DF_PIPELINE_PATH)

### Load fitted pipeline

In [10]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(DF_PIPELINE_PATH)

### Transform dataset using pipeline

In [11]:
transformed_df = pipeline_model \
    .transform(df) \
    .selectExpr('id as id', 'features as features', '_c0 as label')
transformed_df.take(1)

[Row(id=29, features=SparseVector(1957, {0: 3.0, 2: 4.0, 3: 13.0, 4: 224.0, 5: 28.0, 6: 3.0, 7: 35.0, 8: 27.0, 9: 1.0, 10: 1.0, 12: 13.0, 13: 1.0, 1432: 1.0}), label=1)]

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [12]:
import math


def split_by_col(df, split_col, parts_fractions):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    N = df.count()
    ids_sample = df \
        .select(split_col) \
        .sample(False, 0.01) \
        .collect()
    N_sample = len(ids_sample)
    print("Ids size is:", N)
    print("Ids sample size is:", N_sample)
    
    parts = []
    ids_sample = sorted(ids_sample)
    cumulative_part = 0.0
    before_id = None
    for next_part in parts_fractions:
        cumulative_part += next_part
        next_id_index = math.ceil(N_sample * cumulative_part)
        if next_id_index >= N_sample:
            next_id = None
        else:
            next_id = ids_sample[next_id_index][0]
        
        part = df
        if before_id is None and next_part is None and next_part * N_samples < 1:
            # Empty dataframe
            part = part.filter(F.col(split_col).isNull())
        else:
            if before_id is not None:
                part = part.filter(F.col(split_col) >= before_id)
            if next_id is not None:
                part = part.filter(F.col(split_col) < next_id)
        parts.append(part)
    
        before_id = next_id     
    return parts

In [13]:
train_df, val_df, test_df = split_by_col(transformed_df, 'id', [0.8, 0.1, 0.1])

Ids size is: 1834709
Ids sample size is: 18336


In [14]:
N = transformed_df.count()

In [15]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7956068237524316, 0.10398542766182539, 0.10040774858574303)

In [42]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics


def _get_score_and_labels(model, df):
    return model \
        .transform(df) \
        .selectExpr('probability as probability', 'label as label') \
        .rdd \
        .map(lambda row: (float(row.probability[1]), float(row.label)))


def _logloss_from_score_and_labels(score_and_labels):
    return score_and_labels \
        .map(lambda score_label: -(
            score_label[1]*np.log(score_label[0])
            + (1.0-score_label[1])*np.log(1.0-score_label[0]))) \
        .mean()
    

def rocauc(model, df):
    score_and_labels = _get_score_and_labels(model, df)
    metrics = BinaryClassificationMetrics(score_and_labels)
    return metrics.areaUnderROC


def logloss(model, df):
    score_and_labels = _get_score_and_labels(model, df)
    logloss = _logloss_from_score_and_labels(score_and_labels)
    return logloss


def ne(model, df):
    score_and_labels = _get_score_and_labels(model, df)
    nominator = _logloss_from_score_and_labels(score_and_labels)
    mean_ctr = score_and_labels.map(lambda row: row[1]).mean()
    denominator = -(mean_ctr*np.log(mean_ctr) + (1-mean_ctr)*np.log(1-mean_ctr))
    return nominator/denominator

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [52]:
from pyspark.ml.classification import LogisticRegression

best_model = None
best_rocauc = None
for regParam in [0.0, 1e-1, 1e-2, 1e-3]:
    for elasticNetParam in [0.0, 1e-1, 1e-2, 1e-3]: 
        for standardization in [True, False]:
            lr = LogisticRegression(
                regParam=regParam, elasticNetParam=elasticNetParam, standardization=standardization)
            lr_model = lr.fit(train_df)
            lr_rocauc = rocauc(lr_model, val_df)
            if best_rocauc is None or best_rocauc < lr_rocauc:
                print("Improved ROCAUC from {} to {} with params:".format(best_rocauc, lr_rocauc))
                print("regParam={} elasticNetParam={} standardization={}".format(
                    regParam, elasticNetParam, standardization))
                best_model = lr_model
                best_rocauc = lr_rocauc

Improved ROCAUC from None to 0.7024240908087696 with params:
regParam=0.0 elasticNetParam=0.0 standardization=True
Improved ROCAUC from 0.7024240908087696 to 0.7024240908087722 with params:
regParam=0.0 elasticNetParam=0.0 standardization=False
Improved ROCAUC from 0.7024240908087722 to 0.7024240908087724 with params:
regParam=0.0 elasticNetParam=0.01 standardization=True
Improved ROCAUC from 0.7024240908087724 to 0.702590587968243 with params:
regParam=0.01 elasticNetParam=0.1 standardization=True
Improved ROCAUC from 0.702590587968243 to 0.7028322080612285 with params:
regParam=0.001 elasticNetParam=0.1 standardization=True


In [53]:
best_model.transform(val_df).first()

Row(id=446677060223, features=SparseVector(1957, {1: 2.0, 2: 16.0, 3: 11.0, 4: 1.0, 5: 128.0, 6: 1.0, 7: 47.0, 8: 180.0, 10: 1.0, 12: 4.0, 13: 1.0, 1407: 1.0}), label=0, rawPrediction=DenseVector([1.7593, -1.7593]), probability=DenseVector([0.8531, 0.1469]), prediction=0.0)

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [54]:
for cur_df in [val_df, test_df]:
    print("rocauc", rocauc(best_model, cur_df))
    print("logloss", logloss(best_model, cur_df))
    print("ne", ne(best_model, cur_df))

rocauc 0.7028322080612296
logloss 0.5266542614101842
ne 0.917777896552764
rocauc 0.7002145947660112
logloss 0.5310748677287841
ne 0.9203894127198535


## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [55]:
TEST_DATA_PATH = '/workspace/data/mlbd-20-ctr-prediction-1'

TEST_TEST_PATH = os.path.join(TEST_DATA_PATH, 'test.csv')

In [56]:
comp_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_TEST_PATH)

In [57]:
comp_df.printSchema()

root
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = true)
 |-- _c28: string (

In [58]:
comp_df = comp_df.fillna(0, subset=num_columns)
transformed_comp_df = pipeline_model.transform(comp_df)
comp_df_prediction = best_model.transform(transformed_comp_df)

In [59]:
comp_df_prediction.take(1)

[Row(_c1=0, _c2=19, _c3=2, _c4=4, _c5=4576, _c6=6, _c7=6, _c8=5, _c9=15, _c10=0, _c11=2, _c12=0, _c13=4, _c14='05db9164', _c15='b961056b', _c16='bc015839', _c17='4120ff84', _c18='4cf72387', _c19='6f6d9be8', _c20='2682a57d', _c21='5b392875', _c22='a73ee510', _c23='3f25d8e4', _c24='f7433a43', _c25='b838627b', _c26='4fce4e51', _c27='b28479f6', _c28='70a601bc', _c29='0c0bfddd', _c30='e5ba7672', _c31='43de85d3', _c32=None, _c33=None, _c34='b64021bd', _c35=None, _c36='32c7478e', _c37='f1a27f66', _c38=None, _c39=None, id=566935904713, _c14_index=0.0, _c15_index=31.0, _c14_vec=SparseVector(1392, {0: 1.0}), _c15_vec=SparseVector(552, {31: 1.0}), features=SparseVector(1957, {1: 19.0, 2: 2.0, 3: 4.0, 4: 4576.0, 5: 6.0, 6: 6.0, 7: 5.0, 8: 15.0, 10: 2.0, 12: 4.0, 13: 1.0, 1436: 1.0}), rawPrediction=DenseVector([1.1504, -1.1504]), probability=DenseVector([0.7596, 0.2404]), prediction=0.0)]

In [60]:
comp_df_prediction = comp_df_prediction.select(F.col('id'), F.col('probability'))
comp_df_prediction.take(1)

[Row(id=566935904713, probability=DenseVector([0.7596, 0.2404]))]

In [61]:
comp_predictions = comp_df_prediction \
    .rdd \
    .map(lambda row: (row.id, row.probability[1])) \
    .collect()

In [62]:
submission_path = os.path.join(TEST_DATA_PATH, 'submission.csv')
with open(submission_path, 'w') as writer:
    writer.write("id,proba\n")
    for row in comp_predictions:
        writer.write(",".join(map(str, row))+"\n")